# Proyecto individual: Sistema de recomendación de películas
​
Este proyecto constará de dos fases: `Ingenieria de datos`, `Modelamiento y evaluación con machine learning`.

### 1. Ingeniería de datos
* Esto incluye la limpieza y transformación de los datos, abordando problemas como:
    * valores faltantes, 
    * datos duplicados y variables irrelevantes,
    * valores anidados,
    * formateo de columnas,
    * nubes de palabras para ver las más frecuentes.
    * a fin de mejorar la calidad del dataset para el modelado.
    * análisis univariado.
    * análisis bivariado y multivariado.
​
### 2. Modelamiento y evaluación con machine learning
* Implementar un modelo de clasificación con aprendizaje supervisado que permita clasificar (**con un algoritmo de coseno de similitud, por ejemplo**) las películas por ... para encontrar una lista de 5 películas similares 

### 1. Análisis exploratorio de datos

#### 1.1. Importación de librerías

In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import datetime

# Mostrar figuras de matplotlib en el entorno de Jupyter Notebook
%matplotlib inline

#### 1.2. Carga y visualización los datos.

In [3]:
df_movies = pd.read_csv('datasets/movies.csv')
df_movies.head(3)

C:\Users\Josue\AppData\Local\Temp\ipykernel_3236\2792449306.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('datasets/movies.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


#### 1.3. Eliminación de filas

In [4]:
# Se eliminan las películas que su idioma original no es en inglés (habria que analizar que porcentaje representan)
df_movies_1 = df_movies[df_movies['original_language'] == 'en']

In [5]:
# Se filtran las filas con valores notna en las columnas 'video', 'status', 'release_date' 
df_movies_1 = df_movies_1[df_movies_1['video'].notna()] 
df_movies_1 = df_movies_1[df_movies_1['status'].notna()] 
df_movies_1 = df_movies_1[df_movies_1['release_date'].notna()] 

#### 1.4. Eliminación de columnas 

In [6]:
df_movies_2 = df_movies_1.drop(['adult', 'belongs_to_collection', 'status', 'genres', 'poster_path', 'homepage', 'imdb_id', 'overview', 'production_companies', 'production_countries', 'original_language', 'runtime', 'spoken_languages', 'tagline', 'original_title', 'video'], axis=1)


In [7]:
df_movies_2.head()

,budget,id,popularity,release_date,revenue,title,vote_average,vote_count
0,30000000,862,21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0
1,65000000,8844,17.015539,1995-12-15,262797249.0,Jumanji,6.9,2413.0
2,0,15602,11.7129,1995-12-22,0.0,Grumpier Old Men,6.5,92.0
3,16000000,31357,3.859495,1995-12-22,81452156.0,Waiting to Exhale,6.1,34.0
4,0,11862,8.387519,1995-02-10,76578911.0,Father of the Bride Part II,5.7,173.0


#### 2.5. Transformación de valores nulos

2.5.1. Primero se encuentra si existen valores nulos o vacíos

In [8]:
# Encontrar valores nulos
valores_nulos = df_movies_2[['budget', 'revenue']].isnull()
print("Valores nulos en el DataFrame:\n", valores_nulos)

Valores nulos en el DataFrame:
        budget  revenue
0       False    False
1       False    False
2       False    False
3       False    False
4       False    False
...       ...      ...
45459   False    False
45460   False    False
45463   False    False
45464   False    False
45465   False    False

[32135 rows x 2 columns]


In [9]:
nulos_budget = df_movies_2['budget'].isnull().sum()
nulos_revenue = df_movies_2['revenue'].isnull().sum()

print(f"Valores nulos en 'budget': {nulos_budget}")
print(f"Valores nulos en 'revenue': {nulos_revenue}")

Valores nulos en 'budget': 0
Valores nulos en 'revenue': 0


2.5.2. Luego se reemplaza por 0 y se verifica que no haya nulos

In [10]:
df_movies_2[['budget', 'revenue']] = df_movies_2[['budget', 'revenue']].fillna(0)

In [11]:
nulos_budget = df_movies_2['budget'].isnull().sum()
nulos_revenue = df_movies_2['revenue'].isnull().sum()

print(f"Valores nulos en 'budget': {nulos_budget}")
print(f"Valores nulos en 'revenue': {nulos_revenue}")

Valores nulos en 'budget': 0
Valores nulos en 'revenue': 0


#### 1.4. Distinción entre day, month y year en date

In [12]:
# Convertir la columna 'date' a datetime
df_movies_2['release_date'] = pd.to_datetime(df_movies_2['release_date'], errors='coerce')

# Verificar el formato de la columna 'date' después de la conversión
print(df_movies_2['release_date'].head())

0   1995-10-30
1   1995-12-15
2   1995-12-22
3   1995-12-22
4   1995-02-10
Name: release_date, dtype: datetime64[ns]


In [13]:
# Crear nuevas columnas para día, mes y año
df_movies_2['day'] = df_movies_2['release_date'].dt.day
df_movies_2['month'] = df_movies_2['release_date'].dt.month
df_movies_2['release_year'] = df_movies_2['release_date'].dt.year

In [14]:
df_movies_2.head(2)

,budget,id,popularity,release_date,revenue,title,vote_average,vote_count,day,month,release_year
0,30000000,862,21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0,30,10,1995
1,65000000,8844,17.015539,1995-12-15,262797249.0,Jumanji,6.9,2413.0,15,12,1995


In [15]:
df_movies_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 0 to 45465
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   budget        32135 non-null  object        
 1   id            32135 non-null  object        
 2   popularity    32135 non-null  object        
 3   release_date  32135 non-null  datetime64[ns]
 4   revenue       32135 non-null  float64       
 5   title         32135 non-null  object        
 6   vote_average  32135 non-null  float64       
 7   vote_count    32135 non-null  float64       
 8   day           32135 non-null  int32         
 9   month         32135 non-null  int32         
 10  release_year  32135 non-null  int32         
dtypes: datetime64[ns](1), float64(3), int32(3), object(4)
memory usage: 2.6+ MB


In [16]:
df_movies_2.describe()

,release_date,revenue,vote_average,vote_count,day,month,release_year
count,32135,3.213500e+04,32135.000000,32135.000000,32135.000000,32135.000000,32135.000000
mean,1991-08-10 06:16:46.229967360,1.523424e+07,5.500386,142.134744,14.120492,6.451066,1991.117037
min,1878-06-14 00:00:00,0.000000e+00,0.000000,0.000000,1.000000,1.000000,1878.000000
25%,1978-04-06 00:00:00,0.000000e+00,5.000000,3.000000,6.000000,3.000000,1978.000000
50%,2000-12-22 00:00:00,0.000000e+00,5.900000,10.000000,14.000000,7.000000,2000.000000
75%,2010-10-08 12:00:00,0.000000e+00,6.700000,43.000000,22.000000,10.000000,2010.000000
max,2020-12-16 00:00:00,2.787965e+09,10.000000,14075.000000,31.000000,12.000000,2020.000000
std,NaN,7.572686e+07,1.927359,575.694965,9.259583,3.597951,24.724798


#### 1.6. Eliminación de filas con las películas anteriores a 1980, eliminando así más del 25 % de los datos

In [17]:
df_movies_2 = df_movies_2[df_movies_2['release_year'] >= 1990]

In [18]:
df_movies_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20909 entries, 0 to 45465
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   budget        20909 non-null  object        
 1   id            20909 non-null  object        
 2   popularity    20909 non-null  object        
 3   release_date  20909 non-null  datetime64[ns]
 4   revenue       20909 non-null  float64       
 5   title         20909 non-null  object        
 6   vote_average  20909 non-null  float64       
 7   vote_count    20909 non-null  float64       
 8   day           20909 non-null  int32         
 9   month         20909 non-null  int32         
 10  release_year  20909 non-null  int32         
dtypes: datetime64[ns](1), float64(3), int32(3), object(4)
memory usage: 1.7+ MB


## 2.1. Creación de la funcion para el endpoint1

In [19]:
month_mapping = {
    "enero": 1, "febrero": 2, "marzo": 3, "abril": 4,
    "mayo": 5, "junio": 6, "julio": 7, "agosto": 8,
    "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12
}

In [20]:
def cantidad_filmaciones_mes(df, month: str):
    """Obtiene la cantidad de filmaciones según mes indicado
    Args:
        df: DataFrame de películas 
        month (str): nombre del mes en castellano
    Raises:
        HTTPException: en caso de que el mes indicado no sea válido o no esté escrito en castellano o sea nulo
    Returns:
        _type_: entero o integer
    """
    # Convertir el nombre del mes a su número correspondiente
    month_number = month_mapping.get(month.lower())
    if month_number is None:
        raise ValueError(f"Mes '{month}' no es válido.")
    # Filtrar las filas que corresponden al mes especificado
    filtered_df = df[df['month'] == month_number]
    # Contar los IDs en las filas filtradas
    count_ids = filtered_df['id'].count()
    return {"endpoint1": f"Fueron estrenadas {count_ids} peliculas en el mes de {month}"}

In [21]:
cantidad_filmaciones_mes(df=df_movies_2, month='enero')

{'endpoint1': 'Fueron estrenadas 2821 peliculas en el mes de enero'}

## 2.2. Creación de la funcion para el endpoint2

In [22]:
# Función para obtener el día de la semana
def obtener_dia_semana(fecha):
    dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
    if pd.notnull(fecha):
        return dias_semana[int(fecha.day_of_week)]
    else:
        return 'fecha inválida'

# Aplicar la función a la columna 'release_date'
df_movies_2['weekday'] = df_movies_2['release_date'].apply(obtener_dia_semana)

# Se eliminan las filas con valor de celda 'fecha inválida'
df_movies_2 = df_movies_2[df_movies_2['weekday'] != 'fecha inválida']

In [23]:
df_movies_2.head(3)

,budget,id,popularity,release_date,revenue,title,vote_average,vote_count,day,month,release_year,weekday
0,30000000,862,21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0,30,10,1995,lunes
1,65000000,8844,17.015539,1995-12-15,262797249.0,Jumanji,6.9,2413.0,15,12,1995,viernes
2,0,15602,11.7129,1995-12-22,0.0,Grumpier Old Men,6.5,92.0,22,12,1995,viernes


In [24]:
df_movies_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20909 entries, 0 to 45465
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   budget        20909 non-null  object        
 1   id            20909 non-null  object        
 2   popularity    20909 non-null  object        
 3   release_date  20909 non-null  datetime64[ns]
 4   revenue       20909 non-null  float64       
 5   title         20909 non-null  object        
 6   vote_average  20909 non-null  float64       
 7   vote_count    20909 non-null  float64       
 8   day           20909 non-null  int32         
 9   month         20909 non-null  int32         
 10  release_year  20909 non-null  int32         
 11  weekday       20909 non-null  object        
dtypes: datetime64[ns](1), float64(3), int32(3), object(5)
memory usage: 1.8+ MB


##### 2.2.1. Aclaración
Se agrega la columna weekday al dataset porque la memoria usada no se casi incrementada, pasando de 2.1 a 2.2 mb

In [25]:
data_movies = df_movies_2.to_csv('dataset/data_movies.csv')

In [26]:
df = pd.read_csv('dataset/data_movies.csv')

In [27]:
from fastapi import HTTPException

def cantidad_filmaciones_dia(df, weekday: str):
    """Obtiene la cantidad de filmaciones según día de la semana indicado
    Args:
        df: DataFrame de películas 
        weekday (str): nombre del día de la semana en castellano
    Raises:
        HTTPException: en caso de que el día de la semana indicado no sea válido o no esté escrito en castellano o sea nulo
    Returns:
        _type_: entero o integer
    """
    # Lista de días válidos en castellano
    dias_validos = ['lunes', 'martes', 'miércoles',
                    'jueves', 'viernes', 'sábado', 'domingo']
    if weekday not in dias_validos:
        raise HTTPException(
            status_code=400, detail=f"El día '{weekday}' no es válido.")
    # Filtrar las filas que corresponden al dia de la semana especificado
    df_filter_day = df[df['weekday'] == weekday]
    # Contar los IDs en las filas filtradas
    count_ids = df_filter_day['id'].count()
    return {"endpoint2": f"Fueron estrenadas {count_ids} peliculas el dia {weekday}"}

In [28]:
lunes = cantidad_filmaciones_dia(df, 'domingo')
lunes

{'endpoint2': 'Fueron estrenadas 1873 peliculas el dia domingo'}

## 2.3. Creación de la funcion para el endpoint3

In [31]:
from fastapi import HTTPException

def score_titulo(df, titulo_de_la_filmacion: str): 
    """Obtiene el año de estreno y la popularidad de la película ingresada

    Args:
        df: DataFrame de películas
        titulo_de_la_filmacion (str): nombre de la película
    Raises:
        HTTPException: en caso de que la película no esté escrita de forma correcta o no esté ingresada en el dataset o sea inexistente.
    Returns:
        _type_: str
    """
    # Verificar si el título de la película está en el DataFrame
    if titulo_de_la_filmacion not in df['title'].values:
        raise HTTPException(
            status_code=400, detail=f"La película '{titulo_de_la_filmacion}' no ha sido estrenada o no está registrada en el dataset o es inexistente.")
    
    # Filtrar la fila que corresponde a la película especificada
    pelicula = df[df['title'] == titulo_de_la_filmacion]
    
    # Extrar año de estreno y popularidad
    release_year = pelicula['release_year'].values[0]
    popularity = pelicula['popularity'].values[0]
    return {'endpoint3': f"La película {titulo_de_la_filmacion} fue estrenada en el año {release_year} con una popularidad de {popularity}"}

In [32]:
endpoint3 = score_titulo(df, 'Toy Story')
print(endpoint3)

{'endpoint3': 'La película Toy Story fue estrenada en el año 1995 con una popularidad de 21.946943'}
